# Задание 2: Извлечение коллокаций

Выполнено на мощностях [Google Colab](https://colab.research.google.com/drive/1I1QRHbF6W---8LkzfR1ldpDiMcyLj1zs)





## Пункт 1

*Скачайте корпус текстов, обработайте его с помощью UDPipe, извлеките все группы "глагол + прямое дополнение, выраженное существительным" (не учитывайте глаголы, которые встречаются в корпусе менее 50 раз).*

#### Поставим необходимые зависимости:

In [1]:
!wget https://raw.githubusercontent.com/sjut/HSE-Compling/master/hw/testset2.txt

--2019-12-17 03:54:45--  https://raw.githubusercontent.com/sjut/HSE-Compling/master/hw/testset2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1887901 (1.8M) [text/plain]
Saving to: ‘testset2.txt.2’

testset2.txt.2      100%[===================>]   1.80M  --.-KB/s    in 0.09s   

2019-12-17 03:54:45 (20.5 MB/s) - ‘testset2.txt.2’ saved [1887901/1887901]



In [2]:
!wget https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-2998/russian-syntagrus-ud-2.4-190531.udpipe?sequence=74&amp;isAllowed=y
!mv russian-syntagrus-ud-2.4-190531.udpipe?sequence=74 russian-syntagrus-ud-2.4-190531.udpipe

/bin/bash: amp: command not found
--2019-12-17 03:54:50--  https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-2998/russian-syntagrus-ud-2.4-190531.udpipe?sequence=74
Resolving lindat.mff.cuni.cz (lindat.mff.cuni.cz)... 195.113.20.140
Connecting to lindat.mff.cuni.cz (lindat.mff.cuni.cz)|195.113.20.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45859472 (44M) [application/octet-stream]
Saving to: ‘russian-syntagrus-ud-2.4-190531.udpipe?sequence=74’

russian-syntagrus-u 100%[===================>]  43.73M  15.8MB/s    in 2.8s    

2019-12-17 03:54:53 (15.8 MB/s) - ‘russian-syntagrus-ud-2.4-190531.udpipe?sequence=74’ saved [45859472/45859472]



In [3]:
!pip install ufal.udpipe

#### Теперь приступим к выполнению пункта:

Создадим объект UDPipe:

In [0]:
import ufal.udpipe

class UDPipeModel:
    def __init__(self, path):
        """Load given model."""
        self.model = ufal.udpipe.Model.load(path)
        if not self.model:
            raise Exception("Cannot load UDPipe model from file '%s'" % path)

    def tokenize(self, text):
        """Tokenize the text and return list of ufal.udpipe.Sentence-s."""
        tokenizer = self.model.newTokenizer(self.model.DEFAULT)
        if not tokenizer:
            raise Exception("The model does not have a tokenizer")
        return self._read(text, tokenizer)

    def read(self, text, in_format):
        """Load text in the given format (conllu|horizontal|vertical) and return list of ufal.udpipe.Sentence-s."""
        input_format = ufal.udpipe.InputFormat.newInputFormat(in_format)
        if not input_format:
            raise Exception("Cannot create input format '%s'" % in_format)
        return self._read(text, input_format)

    def _read(self, text, input_format):
        input_format.setText(text)
        error = ufal.udpipe.ProcessingError()
        sentences = []

        sentence = ufal.udpipe.Sentence()
        while input_format.nextSentence(sentence, error):
            sentences.append(sentence)
            sentence = ufal.udpipe.Sentence()
        if error.occurred():
            raise Exception(error.message)

        return sentences

    def tag(self, sentence):
        """Tag the given ufal.udpipe.Sentence (inplace)."""
        self.model.tag(sentence, self.model.DEFAULT)

    def parse(self, sentence):
        """Parse the given ufal.udpipe.Sentence (inplace)."""
        self.model.parse(sentence, self.model.DEFAULT)

    def write(self, sentences, out_format):
        """Write given ufal.udpipe.Sentence-s in the required format (conllu|horizontal|vertical)."""

        output_format = ufal.udpipe.OutputFormat.newOutputFormat(out_format)
        output = ''
        for sentence in sentences:
            output += output_format.writeSentence(sentence)
        output += output_format.finishDocument()

        return output

Пришло время разметить текст:

In [0]:
with open("testset2.txt", "r", encoding="utf-8") as intext:
  text = intext.read()

In [6]:
%%time

model = UDPipeModel('russian-syntagrus-ud-2.4-190531.udpipe')
sentences = model.tokenize(text)
for s in sentences:
    model.tag(s)
    model.parse(s)
conllu = model.write(sentences, "conllu")

CPU times: user 3min 9s, sys: 531 ms, total: 3min 9s
Wall time: 3min 9s


Как выглядит наш `conllu`?



In [7]:
print(conllu[:1000])

# newdoc
# newpar
# sent_id = 1
# text = "Я предложил бы "Единой России" объединить в одном окне не только депутатов, но и суд, и прокуратуру, и милицию, и собес,— заявил "Ъ" член политсовета СПС Борис Надеждин.—
1	"	"	PUNCT	_	_	3	punct	_	SpacesBefore=\n\t|SpaceAfter=No
2	Я	я	PRON	_	Case=Nom|Number=Sing|Person=1	3	nsubj	_	_
3	предложил	предложить	VERB	_	Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	0	root	_	_
4	бы	бы	AUX	_	_	3	aux	_	_
5	"	"	PUNCT	_	_	3	punct	_	SpaceAfter=No
6	Единой	единый	ADJ	_	Case=Gen|Degree=Pos|Gender=Fem|Number=Sing	7	amod	_	_
7	России	Россия	PROPN	_	Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing	9	obl	_	SpaceAfter=No
8	"	"	PUNCT	_	_	9	punct	_	_
9	объединить	объединить	VERB	_	Aspect=Perf|VerbForm=Inf|Voice=Act	3	xcomp	_	_
10	в	в	ADP	_	_	12	case	_	_
11	одном	один	NUM	_	Case=Loc|Gender=Neut	12	nummod	_	_
12	окне	окно	NOUN	_	Animacy=Inan|Case=Loc|Gender=Neut|Number=Sing	9	obl	_	_
13	не	не	PART	_	_	14	advmod	_	_
14	только	только	PART	_	_

Посчитаем частотность слов

In [0]:
from collections import Counter

def getWordFrequency(conllu_obj):  # imperfect function but okay for this task
  suitstrs = [s for s in conllu_obj.split("\n") if s]
  suitstrs = [s for s in suitstrs if s[0] in "0123456789"]
  words = [(s.split("\t")[2], s.split("\t")[3]) for s in suitstrs]
  return Counter(words)

In [9]:
%%time

word_count = getWordFrequency(conllu)

CPU times: user 331 ms, sys: 52 ms, total: 383 ms
Wall time: 384 ms


Определим все глаголы, встречающиеся чаще, чем 50 раз:

In [10]:
%%time

suitable_verbs = []
iter_words = iter(word_count.most_common(len(word_count)))
el = next(iter_words)

while el[1] >= 50:
  if el[0][1] == "VERB":
    suitable_verbs.append(el[0])
  el = next(iter_words)

suitable_verbs = set([el[0] for el in suitable_verbs])

CPU times: user 5.68 ms, sys: 0 ns, total: 5.68 ms
Wall time: 5.53 ms


Наконец, выберем все подходящие пары слов:

In [0]:
def getWordPairs(conllu_obj, suitable_verbs):
  spl = conllu_obj.split("\n")
  strpairs = [(spl[i], spl[i+1]) for i, _ in enumerate(spl[:-1]) if spl[i] and spl[i+1]]
  strpairs = [tpl for tpl in strpairs if tpl[0][0] in "0123456789" and tpl[1][0] in "0123456789"]
  wordpairs = [(tpl[0].split("\t")[2], tpl[1].split("\t")[2]) for tpl in strpairs
               if tpl[0].split("\t")[3] == "VERB" and tpl[0].split("\t")[2] in suitable_verbs
               and tpl[1].split("\t")[3] == "NOUN"]
  return wordpairs

In [12]:
%%time

bigrams = getWordPairs(conllu, suitable_verbs)

CPU times: user 251 ms, sys: 2.99 ms, total: 254 ms
Wall time: 257 ms


Вот, что у нас получилось:

In [13]:
bigrams[:10]

[('рассматривать', 'дело'),
 ('рассмотреть', 'вопрос'),
 ('начаться', 'слушание'),
 ('получить', 'срок'),
 ('обвинить', 'олигарх'),
 ('передать', 'судья'),
 ('просить', 'суд'),
 ('рассматривать', 'ходатайство'),
 ('обжаловать', 'постановление'),
 ('сообщить', 'источник')]

## Пункт 2

*Оцените полученные словосочетания следующими метриками: log-likelihood, dice, PMI (можно использовать `nltk.collocations`).*

In [0]:
from nltk.collocations import BigramAssocMeasures
from nltk.collocations import *

bigram_measures = BigramAssocMeasures()

Получим объект со всеми биграммами:

In [0]:
def getBigrams(conllu_obj):
  suitstrs = [s for s in conllu_obj.split("\n") if s]
  suitstrs = [s for s in suitstrs if s[0] in "0123456789"]
  words = [s.split("\t")[2] for s in suitstrs]
  text = words
  return BigramCollocationFinder.from_words(text)

In [19]:
%%time

bcf = getBigrams(conllu)

CPU times: user 599 ms, sys: 23 ms, total: 622 ms
Wall time: 626 ms


Теперь оценим биграммы:

In [0]:
def getBigramScoreList(bcf, relevant_bigrams, score_func):
  scores = bcf.score_ngrams(score_func)
  return_list = [el for el in scores if el[0] in relevant_bigrams]
  return return_list

In [21]:
%%time

log_score = getBigramScoreList(bcf, bigrams, bigram_measures.likelihood_ratio)
pmi_score = getBigramScoreList(bcf, bigrams, bigram_measures.pmi)
dice_score = getBigramScoreList(bcf, bigrams, bigram_measures.dice)

CPU times: user 11.1 s, sys: 2 ms, total: 11.1 s
Wall time: 11.1 s


Что получилось?

In [22]:
log_score[:5]

[(('подать', 'иск'), 413.1231548904059),
 (('удовлетворить', 'иск'), 387.0876918221254),
 (('принять', 'решение'), 314.31659837989565),
 (('удовлетворить', 'ходатайство'), 237.79159038781216),
 (('вынести', 'решение'), 219.65351169722325)]

In [23]:
dice_score[:5]

[(('удовлетворить', 'ходатайство'), 0.17142857142857143),
 (('пройти', 'прения'), 0.17073170731707318),
 (('начаться', 'слушание'), 0.14666666666666667),
 (('подать', 'иск'), 0.14628297362110312),
 (('удовлетворить', 'иск'), 0.14188267394270124)]

In [24]:
pmi_score[:5]

[(('оспаривать', 'выражение'), 11.599301605501264),
 (('принадлежать', 'экс-премьера'), 11.599301605501264),
 (('принадлежать', 'эксглава'), 11.599301605501264),
 (('рассказать', 'обозреватель'), 11.599301605501264),
 (('запретить', 'организацией|вчер'), 11.546834185607128)]

## Пункт 3

Подготовьте "золотой стандарт" коллокаций (далее ЗС) для этого корпуса: возьмите словосочетания, которые попадают в топ-100 по всем метрикам, пересеките со словарем глагольной сочетаемости.

#### Загрузим и предобработаем словарь:

In [25]:
!wget https://raw.githubusercontent.com/isikus/uni-nlp-homeworks/master/hw2/verb_coll.txt

--2019-12-17 04:00:07--  https://raw.githubusercontent.com/isikus/uni-nlp-homeworks/master/hw2/verb_coll.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1083870 (1.0M) [text/plain]
Saving to: ‘verb_coll.txt.1’

verb_coll.txt.1     100%[===================>]   1.03M  --.-KB/s    in 0.07s   

2019-12-17 04:00:07 (14.0 MB/s) - ‘verb_coll.txt.1’ saved [1083870/1083870]



Теперь лемматизируем коллокации в словаре так же, как и в исходном тексте, и положим их в `set`:

In [0]:
with open("verb_coll.txt", "r", encoding="utf-8") as intext:
  text2 = intext.read()

lines = [line for line in text2.split("\n") if line]
lines = [line.split("\t")[-1].capitalize()+"." for line in lines]
text2 = "\n".join(lines)

In [27]:
%%time

sentences2 = model.tokenize(text2)
for s in sentences2:
    model.tag(s)
    model.parse(s)
conllu2 = model.write(sentences2, "conllu")

CPU times: user 20.3 s, sys: 29 ms, total: 20.3 s
Wall time: 20.3 s


In [28]:
print(conllu2[:1000])

# newdoc
# newpar
# sent_id = 1
# text = Аборт делать.
1	Аборт	Аборт	NOUN	_	Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing	2	obj	_	_
2	делать	делать	VERB	_	Aspect=Imp|VerbForm=Inf|Voice=Act	0	root	_	SpaceAfter=No
3	.	.	PUNCT	_	_	2	punct	_	SpacesAfter=\n

# sent_id = 2
# text = Делать аборт.
1	Делать	делать	VERB	_	Aspect=Imp|VerbForm=Inf|Voice=Act	0	root	_	_
2	аборт	аборт	NOUN	_	Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing	1	obj	_	SpaceAfter=No
3	.	.	PUNCT	_	_	1	punct	_	SpacesAfter=\n

# sent_id = 3
# text = Сделать аборт.
1	Сделать	делать	VERB	_	Aspect=Perf|VerbForm=Inf|Voice=Act	0	root	_	_
2	аборт	аборт	NOUN	_	Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing	1	obj	_	SpaceAfter=No
3	.	.	PUNCT	_	_	1	punct	_	SpacesAfter=\n

# sent_id = 4
# text = Авария произошла.
1	Авария	авария	NOUN	_	Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing	2	nsubj	_	_
2	произошла	произойти	VERB	_	Aspect=Perf|Gender=Fem|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	0	root	_	SpaceAfter=No
3	.	.	PUNCT	_	_	2	pu

In [0]:
import re

In [30]:
%%time

dict_pairs = []
relevant_pieces = re.findall(r"\n1.*?\n\n", conllu2, flags=re.DOTALL)

for piece in relevant_pieces:
  piece_lines = [line for line in piece.split("\n") if line]
  piece_lines = [line for line in piece_lines if line[0] in "0123456789"][:-1]
  if len(piece_lines) == 2:
    lemm1 = piece_lines[0].split("\t")[2].lower()
    pos1 = piece_lines[0].split("\t")[3]
    lemm2 = piece_lines[1].split("\t")[2].lower()
    pos2 = piece_lines[1].split("\t")[3]
    if (pos1, pos2) == ("VERB", "NOUN"):
      dict_pairs.append((lemm1, lemm2))
    elif (pos2, pos1) == ("NOUN", "VERB"):
      dict_pairs.append((lemm2, lemm1))

dict_pairs = set(dict_pairs)

CPU times: user 81.6 ms, sys: 997 µs, total: 82.6 ms
Wall time: 82.9 ms


Отлично - теперь у нас есть список устойчивых коллокаций из словаря, лемматизированных так же, как и исходный текст:

In [31]:
list(dict_pairs)[:10]

[('освоить', 'выпуск'),
 ('обеспечить', 'развитие'),
 ('учиться', 'грамота'),
 ('поднять', 'восстание'),
 ('определить', 'влияние'),
 ('увлечься', 'работа'),
 ('слышаться', 'смех'),
 ('продлить', 'срок'),
 ('разработать', 'программа'),
 ('вызывать', 'восторг')]

#### Продолжим работу над пунктом:

Давайте определим, какие биграммы у нас попали во все топ-100 листов:

In [32]:
%%time

top_log = set([el[0] for el in log_score[:100]])
top_pmi = set([el[0] for el in pmi_score[:100]])
top_dice = set([el[0] for el in dice_score[:100]])

golden_top = top_log & top_pmi & top_dice

CPU times: user 715 µs, sys: 0 ns, total: 715 µs
Wall time: 723 µs


Сколько у нас слов в топе?

In [33]:
len(golden_top)

17

Маловато. Тогда давайте нормализуем полученные скоры и определим суммарный топ-100 по трём метрикам:

In [34]:
%%time

top_log_norm = {el[0]: el[1]/log_score[0][1] for el in log_score[:100]}
top_pmi_norm = {el[0]: el[1]/pmi_score[0][1] for el in pmi_score[:100]}
top_dice_norm = {el[0]: el[1]/dice_score[0][1] for el in dice_score[:100]}

top_bigrams = set(top_log_norm.keys())
top_bigrams.update(set(top_pmi_norm.keys()))
top_bigrams.update(set(top_dice_norm.keys()))

golden_top_dict = {}

for bigram in top_bigrams:
  golden_top_dict[bigram] = 0
  if bigram in top_log_norm:
    golden_top_dict[bigram] += top_log_norm[bigram]
  if bigram in top_pmi_norm:
    golden_top_dict[bigram] += top_pmi_norm[bigram]
  if bigram in top_dice_norm:
    golden_top_dict[bigram] += top_dice_norm[bigram]

golden_top = sorted(golden_top_dict.items(), key=lambda x: x[1], reverse=True)[:100]

CPU times: user 1.1 ms, sys: 0 ns, total: 1.1 ms
Wall time: 1.15 ms


Что получилось теперь?

In [35]:
golden_top[:10]

[(('пройти', 'прения'), 2.0126849228249295),
 (('подать', 'иск'), 1.8533173461231014),
 (('удовлетворить', 'иск'), 1.7646278618036844),
 (('подтвердить', 'законность'), 1.6836003108215012),
 (('удовлетворить', 'ходатайство'), 1.575594922659065),
 (('оспорить', 'предписание'), 1.568093467659748),
 (('дать', 'киллер'), 1.507342806760306),
 (('провести', 'обыск'), 1.4770851119565789),
 (('выплатить', 'иркутянин'), 1.424066962634174),
 (('выплатить', 'штрафа'), 1.424066962634174)]

Отлично, это можно пересечь со словарём сочетаемости!

In [0]:
golden_top_intersect = []

for el in golden_top:
  if el[0] in dict_pairs:
    golden_top_intersect.append(el)

Сколько получилось элементов, и какие же они?

In [37]:
print(len(golden_top_intersect))
golden_top_intersect

17


[(('подать', 'иск'), 1.8533173461231014),
 (('удовлетворить', 'иск'), 1.7646278618036844),
 (('провести', 'обыск'), 1.4770851119565789),
 (('дать', 'согласие'), 1.3392317276793677),
 (('принять', 'решение'), 1.2891321492919685),
 (('объявить', 'перерыв'), 1.136891001856666),
 (('вынести', 'приговор'), 0.9721202463246261),
 (('делать', 'акцент'), 0.9558681155175195),
 (('вынести', 'решение'), 0.9389228532044955),
 (('предъявить', 'обвинение'), 0.9259952139276855),
 (('дать', 'добро'), 0.8815104942068533),
 (('дать', 'пощечин'), 0.8815104942068533),
 (('принять', 'резолюция'), 0.8786214646991715),
 (('обжаловать', 'решение'), 0.8758550830228828),
 (('принять', 'ислам'), 0.8704663098150166),
 (('рассмотреть', 'жалоба'), 0.831168968798167),
 (('отменить', 'постановление'), 0.7615319443352115)]

## Пункт 4

*Добавьте в ЗС словосочетания из топ-100 , которые не вошли в словарь, но являются коллокациями (если такие есть), объясните свой выбор.*

Давайте посмотрим на 83 невошедших коллокации:

In [42]:
out_of_intersect = [el[0] for el in golden_top if el not in golden_top_intersect]

for i, el in enumerate(out_of_intersect):
  print(str(i)+".", " ".join(el), sep="\t")

0.	пройти прения
1.	подтвердить законность
2.	удовлетворить ходатайство
3.	оспорить предписание
4.	дать киллер
5.	выплатить иркутянин
6.	выплатить штрафа
7.	рассказать собеседник
8.	иметь право
9.	обязать горсовет
10.	пояснить собеседник
11.	рассказать обозреватель
12.	оспаривать выражение
13.	принадлежать эксглава
14.	принадлежать экс-премьера
15.	обязать экс-премьер
16.	запретить организацией|вчер
17.	запретить телекритика
18.	оспорить правомерность
19.	дать генпрокуратура
20.	пройти серия
21.	принадлежать экссенатор
22.	подтвердить правомочность
23.	начаться слушание
24.	использовать название
25.	выплатить хоккеист
26.	рассказать миноритарий
27.	рассматривать прошение
28.	подтвердить правомерность
29.	доказать правность
30.	использовать женщинсмертница
31.	использовать водомет
32.	оказаться втянутый
33.	оказаться козлый
34.	провести демонстрация
35.	дать наказ
36.	дать бог
37.	дать отмашка
38.	утверждать очевидец
39.	объявить персона
40.	говорить PR
41.	арестовывать заочно|Как
42.	и

Выберем такие биграммы, слова в которых значительно чаще встречаются друг с другом, чем с основной массой других и/или образуют вместе особый контекст:

In [0]:
add_to_collocations = [2, 6, 8, 36, 37, 59, 65, 70, 71, 81]

In [0]:
final_golden = [el[0] for el in golden_top_intersect]
for i in add_to_collocations:
  final_golden.append(out_of_intersect[i])

In [45]:
final_golden

[('подать', 'иск'),
 ('удовлетворить', 'иск'),
 ('провести', 'обыск'),
 ('дать', 'согласие'),
 ('принять', 'решение'),
 ('объявить', 'перерыв'),
 ('вынести', 'приговор'),
 ('делать', 'акцент'),
 ('вынести', 'решение'),
 ('предъявить', 'обвинение'),
 ('дать', 'добро'),
 ('дать', 'пощечин'),
 ('принять', 'резолюция'),
 ('обжаловать', 'решение'),
 ('принять', 'ислам'),
 ('рассмотреть', 'жалоба'),
 ('отменить', 'постановление'),
 ('удовлетворить', 'ходатайство'),
 ('выплатить', 'штрафа'),
 ('иметь', 'право'),
 ('дать', 'бог'),
 ('дать', 'отмашка'),
 ('предъявить', 'ультиматум'),
 ('выплатить', 'компенсация'),
 ('подать', 'апелляция'),
 ('делать', 'замечание'),
 ('объявить', 'голодовка')]